<div align=center>
		
<p></p>
<p></p>
<font size=5>
به نام خدا
<font/>
<p></p>
<p></p>
<font color=green size=4>
بازیابی پیشرفته اطلاعات - تمرین سوم
<br/>
بوستان و گلستان سعدی
</font>
<br/>
<br/>
<font size=3>
بهار 1401
<br/>
<font/>
<br/>


</div>

<hr/>
<div align=center>
		    <font size=3>
			    <br />
				گروه 33:
				علی مهربانی ، آرمان سلیمانی ، پانیذ حلواچی
            	<br/>
			</font>
    <br/>
    </div>

In [1]:
#import necessary libraries
!pip install hazm
from __future__ import unicode_literals
from hazm import *
import json
from random import randint
from nltk import FreqDist
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
import codecs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
from gensim.models import FastText

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<div align=center>
		    <font size=4 color=green>
			    <br />
                بخش اول:
                </font>
                <font size=3>
                دریافت داده‌ها
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [2]:
all_files = listdir('Boostan/')

i = 0
df = pd.DataFrame(columns=[0])

for p in all_files:
    f = open('Boostan/' + p, 'r', encoding='utf8')
    
    has_poems = True
    
    
    while has_poems:
        poem = ''
        
        while True:
            s = f.readline()
            if '$' in s:
                has_poems = False
                break
            elif '_' in s:
                break
            poem = poem + s
        
        #inja ye poem kamel shode
        if poem != '':
            df.loc[i] = poem
            i = i + 1

In [3]:
df.size

224

<div align=center>
		    <font size=4 color=green>
			    <br />
                بخش دوم:
                </font>
                <font size=3>
                پیش پردازش اولیه متن
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [4]:
#copy df
poems = df.copy()
print(df.at[223, 0])

جوانی هنرمند فرزانه بود
که در وعظ چالاک و مردانه بود
نکونام و صاحبدل و حق پرست
خط عارضش خوشتر از خط دست
قوی در بلاغات و در نحو چست
ولی حرف ابجد نگفتی درست
یکی را بگفتم ز صاحبدلان
که دندان پیشین ندارد فلان
برآمد ز سودای من سرخ روی
کز این جنس بیهوده دیگر مگوی
تو در وی همان عیب دیدی که هست
ز چندان هنر چشم عقلت ببست
یقین بشنو از من که روز یقین
نبینند بد، مردم نیک بین
یکی را که فضل است و فرهنگ و رای
گرش پای عصمت بخیزد ز جای
به یک خرده مپسند بر وی جفا
بزرگان چه گفتند؟ خذ ما صفا
بود خار و گل با هم ای هوشمند
چه در بند خاری؟ تو گل دسته بند
که را زشت خویی بود در سرشت
نبیند ز طاووس جز پای زشت
صفایی به دست آور ای خیره روی
که ننماید آیینهٔ تیره، روی
طریقی طلب کز عقوبت رهی
نه حرفی که انگشت بر وی نهی
منه عیب خلق ای خردمند پیش
که چشمت فرو دوزد از عیب خویش
چرا دامن آلوده را حد زنم
چو در خود شناسم که تردامنم؟
نشاید که بر کس درشتی کنی
چو خود را به تأویل پشتی کنی
چو بد ناپسند آیدت خود مکن
پس آنگه به همسایه گو بد مکن
من ار حق شناسم وگر خود نمای
برون با تو دارم، درون با خدای
چو ظاهر به عفت بیاراستم
تصرف مکن

### z/az  gah/gaah hazf kardane b az fe'l

In [5]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

for i in range(0, df.shape[0]):
    df.at[i, 0] = normalizer.normalize(df.at[i, 0]).replace(' ز ', ' از ').replace(' مر ', ' ') #normalize beshe va z beshe az
    
    words = word_tokenize(df.at[i, 0])
    
    for j in range(0, len(words)):
        
        word = words[j]
        past_root = lemmatizer.lemmatize(word).split('#', 1)[0]
        past_root = past_root.replace('آ', 'ا')
        word_modified = word.replace(past_root, '')
        
        if word_modified != '':
            if word_modified[0] == 'ب' and len(word_modified) < 4:
                #yani fele maazi bode be forme ghadimi
                df.at[i, 0] = df.at[i, 0].replace(word, word[1:])
        
        if word.endswith('گه'):
            df.at[i, 0] = df.at[i, 0].replace(word, word[:-2]+'گاه')

### fele amre bastani

In [6]:
for i in range(0, df.shape[0]):
    
    words = word_tokenize(df.at[i, 0])
    
    for j in range(0, len(words)):
        
        word = words[j]
        
        if word[0] == 'م':
            word_modified = 'ب' + word[1:]
            present_root = lemmatizer.lemmatize(word_modified).split('#', 1)[-1]
            present_root = present_root.replace('آ', 'ا')
            word_modified = word_modified.replace(present_root, '')
            
            if word_modified == 'ب':
                
                df.at[i, 0] = df.at[i, 0].replace(word, 'ن'+word[1:])

### other preprocess reqs

In [7]:
with open('Exception.txt', encoding='utf8') as exc:
    data = exc.read()
special_verbs = json.loads(data)


df_temp = pd.DataFrame(columns=[0])
stopwords = ['!','،','؟','ز','ار']+[normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

for i in range(0, df.shape[0]):
    
    words = word_tokenize(df.at[i, 0])
    words = [t for t in words if t not in stopwords]
    lemmatized_tokens = []
    
    for t in words:
        if t in special_verbs.keys():
            lemmatized_tokens.append(t)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(t).split('#')[0])
    
    lemmatized = ' '.join(lemmatized_tokens)
            
    df.at[i, 0] = lemmatized
    df_temp.at[i, 0] = lemmatized_tokens



In [8]:
print(df_temp.at[0, 0])

['برآریم', 'دست', 'دل', 'توانست', 'برآورد', 'فردا', 'گل', 'فصل', 'خزان', 'دید', 'درخت', 'برگ', 'ماند', 'سرما', 'سخت', 'برآرد', 'تهی', 'دست', 'رحمت', 'گشت', 'تهیدست', 'مپندار', 'هرگز', 'بست', 'نومید', 'آورده', 'دست', 'قضا', 'خلعت', 'ناندارش', 'قدر', 'میوه', 'آستین', 'نهد', 'طاعت', 'آرند', 'مسکین', 'درگاه', 'مسکین', 'نواز', 'چو', 'شاخ', 'برهنه', 'برآریم', 'دست', 'برگ', 'توانست', 'خداوندگارا', 'کن', 'جود', 'جرم', 'بندگان', 'گناه', 'آمد', 'بنده', 'خاکسار', 'امید', 'عفو', 'خداوندگار', 'کریما', 'رزق', 'پرورده', 'انعام', 'لطف', 'خو', 'کرد', 'گدا', 'کرم', 'دید', 'لطف', 'ناز', 'گشت', 'دنبال', 'بخشنده', 'چو', 'دنیا', 'کردی', 'عزیز', 'عقب', 'چشم', 'عزیزی', 'خوار', 'بس', 'عزیز', 'خوار', 'دید', 'کس', 'خدایا', 'عزت', 'خوار', 'کرد', 'ذل', 'گنه', 'شرمسار', 'کرد', 'مسلط', 'کرد', 'منی', 'سر', 'دست', 'به\u200cگر', 'عقوبت', 'رم', 'گیتی', 'بتر', 'زین', 'بود', 'بدی', 'جفا', 'بردن', 'دست', 'خودی', 'مرا', 'شرمساری', 'بس', 'دگر', 'شرمسار', 'کرد', 'کس', 'گرم', 'سر', 'افتاد', 'سایه', 'سپهر', 'که', 'پایه\u200cای'

<div align=center>
		    <font size=4 color=green>
			    <br />
                بخش سوم:
                </font>
                <font size=3>
                پیشنهاد شعر
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [16]:
#helper functions

#return index of 10 largest elements
def ten_largest_elements(list1,k):
  templist = list1.copy()
  index_list = []
  for i in range(k):
    index, largest_values = largest_element_index(templist)
    index_list.append(index)
    templist[index] = 0
  return index_list

#return the largest element
def largest_element_index(temp_list):
  largest_value = 0
  index = 0
  for i,element in enumerate(temp_list):
    if element>largest_value:
      largest_value = element
      index = i
  return (index,largest_value)

#print poems with mentioned indices
def print_poems(list_of_indices):
  for i, index in enumerate(list_of_indices):
    print("rank ",i+1,":")
    print(poems.at[i,0],"\n")

<div dir="rtl">
		    <font size=2>
			      1- استفاده از روش boolean
			</font>
    <br/>
    </div>

In [10]:
documents = list(itertools.chain(*df.values))
number_of_documents = len(documents)

In [11]:
vectorizer = CountVectorizer()
vectorizer.fit(documents)
vocabulary = vectorizer.vocabulary_
#unique words
terms = list(vocabulary.keys())

In [12]:
#return boolean vector representation 
def bool_Representation(documents):
  matrix = []
  for document in documents:
    dummy_list = []
    for j in terms:
      if j in document:
        dummy_list.append(1)
      else:
        dummy_list.append(0)
    matrix.append(dummy_list)
  return matrix

#return jaccard similarity between two sets
def jaccard_similarity(A, B):
    nominator = A.intersection(B)
    denominator = A.union(B)
    similarity = len(nominator)/len(denominator)
    return similarity

#return number of similar elements between query and each document
def prediction(query_vector):
    scores = []
    count = 0
    term_len = len(terms)
    for i, vector in enumerate(matrix):
        for t in range(term_len):
            if(query_vector[t] == vector[t]):
                count += 1
        scores.append(count)
        count = 0
    return scores

In [17]:
matrix = bool_Representation(documents)
# desired query [does not need a tokenization]
query = bool_Representation([df.at[5, 0]])[0]
###########
scores = prediction(query)
print_poems(ten_largest_elements(scores,10))

rank  1 :
بیا تا برآریم دستی ز دل
که نتوان برآورد فردا ز گل
به فصل خزان در نبینی درخت
که بی برگ ماند ز سرمای سخت
برآرد تهی دستهای نیاز
ز رحمت نگردد تهیدست باز
مپندار از آن در که هرگز نبست
که نومید گردد بر آورده دست
قضا خلعتی نامدارش دهد
قدر میوه در آستینش نهد
همه طاعت آرند و مسکین نیاز
بیا تا به درگاه مسکین نواز
چو شاخ برهنه برآریم دست
که بی برگ از این بیش نتوان نشست
خداوندگارا نظر کن به جود
که جرم آمد از بندگان در وجود
گناه آید از بندهٔ خاکسار
به امید عفو خداوندگار
کریما به رزق تو پرورده‌ایم
به انعام و لطف تو خو کرده‌ایم
گدا چون کرم بیند و لطف و ناز
نگردد ز دنبال بخشنده باز
چو ما را به دنیا تو کردی عزیز
به عقبی همین چشم داریم نیز
عزیزی و خواری تو بخشی و بس
عزیز تو خواری نبیند ز کس
خدایا به عزت که خوارم مکن
به ذل گنه شرمسارم مکن
مسلط مکن چون منی بر سرم
ز دست تو به گر عقوبت برم
به گیتی بتر زین نباشد بدی
جفا بردن از دست همچون خودی
مرا شرمساری ز روی تو بس
دگر شرمسارم مکن پیش کس
گرم بر سر افتد ز تو سایه‌ای
سپهرم بود کهترین پایه‌ای
اگر تاج بخشی سر افرازدم
تو بردار تا کس نیندازدم
تنم می‌بلرز

<div dir="rtl">
		    <font size=2>
			      2- استفاده از tf-idf
			</font>
    <br/>
    </div>

In [18]:
documents = list(itertools.chain(*df.values))
number_of_documents = len(documents)

In [19]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(documents)
tfidf_matrix = tfidf_vectorizer.transform(documents).todense().tolist()
#desired query [does not need tokenization]
query = [df.at[5, 0]]
#################
query_vector = tfidf_vectorizer.transform(query).todense().tolist()[0]
#calculate cosine similarity
similarities = []
for tf_idf_vector in tfidf_matrix:
  similarity = 1 - spatial.distance.cosine(query_vector,tf_idf_vector)
  similarities.append(similarity)

In [20]:
print_poems(ten_largest_elements(similarities,10))

rank  1 :
بیا تا برآریم دستی ز دل
که نتوان برآورد فردا ز گل
به فصل خزان در نبینی درخت
که بی برگ ماند ز سرمای سخت
برآرد تهی دستهای نیاز
ز رحمت نگردد تهیدست باز
مپندار از آن در که هرگز نبست
که نومید گردد بر آورده دست
قضا خلعتی نامدارش دهد
قدر میوه در آستینش نهد
همه طاعت آرند و مسکین نیاز
بیا تا به درگاه مسکین نواز
چو شاخ برهنه برآریم دست
که بی برگ از این بیش نتوان نشست
خداوندگارا نظر کن به جود
که جرم آمد از بندگان در وجود
گناه آید از بندهٔ خاکسار
به امید عفو خداوندگار
کریما به رزق تو پرورده‌ایم
به انعام و لطف تو خو کرده‌ایم
گدا چون کرم بیند و لطف و ناز
نگردد ز دنبال بخشنده باز
چو ما را به دنیا تو کردی عزیز
به عقبی همین چشم داریم نیز
عزیزی و خواری تو بخشی و بس
عزیز تو خواری نبیند ز کس
خدایا به عزت که خوارم مکن
به ذل گنه شرمسارم مکن
مسلط مکن چون منی بر سرم
ز دست تو به گر عقوبت برم
به گیتی بتر زین نباشد بدی
جفا بردن از دست همچون خودی
مرا شرمساری ز روی تو بس
دگر شرمسارم مکن پیش کس
گرم بر سر افتد ز تو سایه‌ای
سپهرم بود کهترین پایه‌ای
اگر تاج بخشی سر افرازدم
تو بردار تا کس نیندازدم
تنم می‌بلرز

<div dir="rtl">
		    <font size=2>
			      3- استفاده از روش fasttext
			</font>
    <br/>
    </div>

In [22]:
#set hyperparameters
embedding_size = 60
window_size = 40
min_word = 5
down_sampling = 1e-2
word_tokenized_corpus = list(itertools.chain(*df_temp.values))
word_tokenizad_temp = word_tokenized_corpus.copy()

In [23]:
# define model and train it
model = FastText(size=embedding_size, window=window_size, min_count=min_word, sample=down_sampling)
model.build_vocab(word_tokenized_corpus)
model.train(word_tokenized_corpus, total_examples=len(word_tokenized_corpus), epochs=100)

In [24]:
#calculate mean vector for each poem
poem_vectors=[[]] * len(word_tokenizad_temp)
for i,poem in enumerate(word_tokenizad_temp):
  poem_vectors[i] = [0 for i in range(embedding_size)]
  length = len(poem)
  for word in poem:
    try:
        temp = model.wv[word]
        vector = poem_vectors[i]
        poem_vectors[i] = [vector[j]+temp[j] for j in range(embedding_size)]
        break
    except:
        length -= 1
  poem_vectors[i] = [item / length for item in poem_vectors[i]]

In [25]:
# desired query [tokenization is needed]
tokenized_query = df_temp.at[5,0]
##############
query_vector = [0 for i in range(embedding_size)]
length = len(tokenized_query)
#calculate mean of query's words vectors
for word1 in tokenized_query:
  try:
        temp1 = model.wv[word1]
        query_vector = [query_vector[j]+temp1[j] for j in range(embedding_size)]
        break
  except:
        length -= 1
query_vector = [item / length for item in query_vector]

In [26]:
#calculate cosine similarity
similarities = []
for poem_vector in poem_vectors:
  similarity = 1 - spatial.distance.cosine(query_vector,poem_vector)
  similarities.append(similarity)

In [27]:
print_poems(ten_largest_elements(similarities,10))

rank  1 :
بیا تا برآریم دستی ز دل
که نتوان برآورد فردا ز گل
به فصل خزان در نبینی درخت
که بی برگ ماند ز سرمای سخت
برآرد تهی دستهای نیاز
ز رحمت نگردد تهیدست باز
مپندار از آن در که هرگز نبست
که نومید گردد بر آورده دست
قضا خلعتی نامدارش دهد
قدر میوه در آستینش نهد
همه طاعت آرند و مسکین نیاز
بیا تا به درگاه مسکین نواز
چو شاخ برهنه برآریم دست
که بی برگ از این بیش نتوان نشست
خداوندگارا نظر کن به جود
که جرم آمد از بندگان در وجود
گناه آید از بندهٔ خاکسار
به امید عفو خداوندگار
کریما به رزق تو پرورده‌ایم
به انعام و لطف تو خو کرده‌ایم
گدا چون کرم بیند و لطف و ناز
نگردد ز دنبال بخشنده باز
چو ما را به دنیا تو کردی عزیز
به عقبی همین چشم داریم نیز
عزیزی و خواری تو بخشی و بس
عزیز تو خواری نبیند ز کس
خدایا به عزت که خوارم مکن
به ذل گنه شرمسارم مکن
مسلط مکن چون منی بر سرم
ز دست تو به گر عقوبت برم
به گیتی بتر زین نباشد بدی
جفا بردن از دست همچون خودی
مرا شرمساری ز روی تو بس
دگر شرمسارم مکن پیش کس
گرم بر سر افتد ز تو سایه‌ای
سپهرم بود کهترین پایه‌ای
اگر تاج بخشی سر افرازدم
تو بردار تا کس نیندازدم
تنم می‌بلرز